# Spread Sheet에서 데이터 가져오기

In [ ]:
import os
import requests
import pandas as pd

WEB_APP_URL = os.getenv("APPS_SCRIPT_WEBAPP_URL", "https://script.google.com/macros/s/AKfycbw5hojwpsNR15W5Wl69HHl5aLg1tG0vxg6XQvYFlsqIeM_QwDsq9wCRSIO7S0y3DBDz6Q/exec")

# 데이터 요청
resp = requests.get(WEB_APP_URL, timeout=30)
resp.raise_for_status()
payload = resp.json()

rows = payload.get("rows", [])
df = pd.DataFrame(rows)

# TimeStamp를 한국 시간(KST)으로 변환해 별도 컬럼 추가
from zoneinfo import ZoneInfo
_src_tz = os.getenv("SOURCE_TZ", "UTC")
_kst = ZoneInfo("Asia/Seoul")

_ts = pd.to_datetime(df["TimeStamp"], errors="coerce", dayfirst=True)

try:
    _ts = _ts.dt.tz_localize(ZoneInfo(_src_tz))
except TypeError:
    pass

# KST로 변환 후 문자열로 표현
_ts = _ts.dt.tz_convert(_kst)
df["TimeStampKST"] = _ts.dt.strftime("%d/%m/%Y %H:%M:%S")

df.head()



,UserID,TimeStamp,TimeKnown,TestAResult,TestBResult,TimeStampKST
0,12,2025-08-12T11:51:30.000Z,5년 이상,"{""pre_a1_뉴욕"":""자유의 여신상"",""pre_a2_뉴욕"":""뉴욕의 가장 대표적...",{},08/12/2025 20:51:30
1,11,2025-08-12T11:58:21.000Z,5년 이상,"{""pre_a1_뉴욕"":""자유의 여신상"",""pre_a2_뉴욕"":""뉴욕하면 바로 생각...",{},08/12/2025 20:58:21
2,12,2025-08-12T12:04:30.000Z,5년 이상,{},"{""recv_angry_o_valence"":""7"",""recv_angry_1_vale...",08/12/2025 21:04:30
3,11,2025-08-12T12:18:45.000Z,5년 이상,{},"{""recv_experience_chamad_known_level_3"":true,""...",08/12/2025 21:18:45


# UserID별 데이터 통합

In [ ]:
df_src = df.copy()

mask_a = df_src['TestAResult'].notna() & (df_src['TestAResult'].astype(str).str.strip() != '')
mask_b = df_src['TestBResult'].notna() & (df_src['TestBResult'].astype(str).str.strip() != '')

cols_a = [c for c in ['UserID', 'TimeKnown', 'TimeStamp', 'TestAResult'] if c in df_src.columns]
cols_b = [c for c in ['UserID', 'TimeKnown', 'TimeStamp', 'TestBResult'] if c in df_src.columns]

df_a = df_src.loc[mask_a, cols_a].copy().rename(columns={'TimeStamp': 'ATimeStampKST'})
df_b = df_src.loc[mask_b, cols_b].copy().rename(columns={'TimeStamp': 'BTimeStampKST'})

df_ab = pd.merge(df_a, df_b, on='UserID', how='outer', suffixes=('', '_b'))

if 'TimeKnown_b' in df_ab.columns:
    if 'TimeKnown' in df_ab.columns:
        df_ab['TimeKnown'] = df_ab['TimeKnown'].combine_first(df_ab['TimeKnown_b'])
        df_ab = df_ab.drop(columns=['TimeKnown_b'])
    else:
        df_ab = df_ab.rename(columns={'TimeKnown_b': 'TimeKnown'})

ordered_cols = [c for c in ['UserID', 'TimeKnown', 'TestAResult', 'TestBResult', 'ATimeStampKST', 'BTimeStampKST'] if c in df_ab.columns]
df_ab = df_ab[ordered_cols]

df_ab.head()


,UserID,TimeKnown,TestAResult,TestBResult,ATimeStampKST,BTimeStampKST
0,11,5년 이상,"{""pre_a1_뉴욕"":""자유의 여신상"",""pre_a2_뉴욕"":""뉴욕하면 바로 생각...",{},2025-08-12T11:58:21.000Z,2025-08-12T11:58:21.000Z
1,11,5년 이상,"{""pre_a1_뉴욕"":""자유의 여신상"",""pre_a2_뉴욕"":""뉴욕하면 바로 생각...","{""recv_experience_chamad_known_level_3"":true,""...",2025-08-12T11:58:21.000Z,2025-08-12T12:18:45.000Z
2,11,5년 이상,{},{},2025-08-12T12:18:45.000Z,2025-08-12T11:58:21.000Z
3,11,5년 이상,{},"{""recv_experience_chamad_known_level_3"":true,""...",2025-08-12T12:18:45.000Z,2025-08-12T12:18:45.000Z
4,12,5년 이상,"{""pre_a1_뉴욕"":""자유의 여신상"",""pre_a2_뉴욕"":""뉴욕의 가장 대표적...",{},2025-08-12T11:51:30.000Z,2025-08-12T11:51:30.000Z


In [ ]:
import json
import pandas as pd

def is_empty_json(val):
    if isinstance(val, dict):
        return len(val) == 0
    if pd.isna(val):
        return False
    s = str(val).strip()
    if s in ("{}", "{ }"):
        return True
    try:
        parsed = json.loads(s)
        return isinstance(parsed, dict) and len(parsed) == 0
    except Exception:
        return False

mask_empty = df_ab["TestAResult"].apply(is_empty_json) | df_ab["TestBResult"].apply(is_empty_json)
df_ab_no_empty = df_ab.loc[~mask_empty].reset_index(drop=True)

df_ab_no_empty.head()

,UserID,TimeKnown,TestAResult,TestBResult,ATimeStampKST,BTimeStampKST
0,11,5년 이상,"{""pre_a1_뉴욕"":""자유의 여신상"",""pre_a2_뉴욕"":""뉴욕하면 바로 생각...","{""recv_experience_chamad_known_level_3"":true,""...",2025-08-12T11:58:21.000Z,2025-08-12T12:18:45.000Z
1,12,5년 이상,"{""pre_a1_뉴욕"":""자유의 여신상"",""pre_a2_뉴욕"":""뉴욕의 가장 대표적...","{""recv_angry_o_valence"":""7"",""recv_angry_1_vale...",2025-08-12T11:51:30.000Z,2025-08-12T12:04:30.000Z


# TestAResult만 추출 후 JSON 파싱 후 컬럼으로 추가

In [ ]:
# df_ab_no_empty → df_a 생성(파싱) + 케이스 분리(df_a_12, df_a_34)
import json, ast
import pandas as pd

def to_dict(v):
    if isinstance(v, dict):
        return v
    if pd.isna(v):
        return {}
    s = str(v).strip()
    if not s:
        return {}
    try:
        return json.loads(s)
    except Exception:
        try:
            return ast.literal_eval(s)
        except Exception:
            return {}

# 필요한 컬럼만 선택
base = df_ab_no_empty[["UserID", "TimeKnown", "ATimeStampKST", "TestAResult"]].copy()

# TestAResult 파싱 → 컬럼 확장
parsed = base["TestAResult"].map(to_dict)
feat = pd.json_normalize(parsed)

# df_a 구성: UserID, TimeKnown, ATimeStampKST + JSON 키 컬럼들
df_a = pd.concat([base.drop(columns=["TestAResult"]), feat], axis=1)

# 케이스 분리: case1,2 → df_a_12 / case3,4 → df_a_34
user_id_num = pd.to_numeric(df_a["UserID"], errors="coerce")
mod = (user_id_num % 4).astype("Int64")

df_a_12 = df_a[mod.isin([1, 2])].reset_index(drop=True)
df_a_34 = df_a[mod.isin([3, 0])].reset_index(drop=True)

In [17]:
df_a_12

,UserID,TimeKnown,ATimeStampKST,pre_a1_뉴욕,pre_a2_뉴욕,pre_a3_뉴욕,pre_a4_뉴욕,pre_a5_뉴욕,pre_a6_뉴욕,post_a1_뉴욕,...,post_a1_파리,post_a2_파리,post_a3_파리,post_a4_파리,post_a5_파리,post_a6_파리,post_a7_파리,post_a8_파리,post_a9_파리,post_a10_파리


In [18]:
df_a_34

,UserID,TimeKnown,ATimeStampKST,pre_a1_뉴욕,pre_a2_뉴욕,pre_a3_뉴욕,pre_a4_뉴욕,pre_a5_뉴욕,pre_a6_뉴욕,post_a1_뉴욕,...,post_a1_파리,post_a2_파리,post_a3_파리,post_a4_파리,post_a5_파리,post_a6_파리,post_a7_파리,post_a8_파리,post_a9_파리,post_a10_파리
0,11,5년 이상,2025-08-12T11:58:21.000Z,자유의 여신상,뉴욕하면 바로 생각나기도 하고 예전부터 실제로 보고싶었기 떄문이다.,빌리지 할로윈 퍼레이드,가장 익숙하고 흥미로운 미국의 문화라 보고싶었기 떄문이다.,"필라델피아의 베지에서 루타바가 퐁뒤, 그릴드세이탄, 플로리다 콘",현지의 신선한 채소를 가지고 만든 맛있는 음식을 먹어보고 싶기 떄문이다. 한국에서 ...,6,...,7,6,6,6,6,7,2,7,7,7
1,12,5년 이상,2025-08-12T11:51:30.000Z,자유의 여신상,뉴욕의 가장 대표적인 관광지고 뉴욕에 들린다면 반드시 한번쯤은 봐야 하는 풍경,빌리지 할로윈 퍼레이드,익히 알고있고 흥미로워하는 문화인데 미국이 할로윈 행사에 열성적이니 한 번 체험해보...,홈메이드 음식 만드는 식당 어딘가,가장 미국스러운 맛을 체험해보고 싶어서,5,...,7,7,7,7,7,4,7,7,7,4


In [21]:
df_a_34.to_excel("df_a_34.xlsx")

# TestBResult만 추출 후 JSON 파싱 후 컬럼으로 추가


In [ ]:
import json, ast

def to_dict(v):
    if isinstance(v, dict):
        return v
    if pd.isna(v):
        return {}
    s = str(v).strip()
    if not s:
        return {}
    try:
        return json.loads(s)
    except Exception:
        try:
            return ast.literal_eval(s)
        except Exception:
            return {}

base_b = df_ab_no_empty[["UserID", "TimeKnown", "BTimeStampKST", "TestBResult"]].copy()
parsed_b = base_b["TestBResult"].map(to_dict)
feat_b = pd.json_normalize(parsed_b)

df_b = pd.concat([base_b.drop(columns=["TestBResult"]), feat_b], axis=1)
df_b = df_b[["UserID", "TimeKnown", "BTimeStampKST"] + [c for c in feat_b.columns]]

df_b.head()

,UserID,TimeKnown,BTimeStampKST,recv_experience_chamad_known_level_3,recv_chamad_o_expression,recv_chamad_o_valence,recv_chamad_2_valence,recv_chamad_o_arousal,recv_chamad_2_arousal,recv_chamad_2_expression,...,send_mad_2_valence,send_mad_2_arousal,send_mad_2_expression,send_satis_2_valence,send_satis_2_arousal,send_satis_2_expression,send_experience_happy_known_level_3,send_happy_2_valence,send_happy_2_arousal,send_happy_2_expression
0,11,5년 이상,2025-08-12T12:18:45.000Z,True,6,2,3,6,7,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12,5년 이상,2025-08-12T12:04:30.000Z,True,7,7,NaN,7,NaN,NaN,...,1,1,1,1,1,6,False,1,1,2


In [20]:
df_b.to_excel("df_b.xlsx")